In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip3 install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.11)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.18)
xz-utils is already installed at the requested version (5.2.2-1.3ubuntu0.1)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-2

In [ ]:
import requests
from bs4 import BeautifulSoup
import sys
import MeCab
from time import sleep
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

##関数定義
# URLからテキスト情報をスクレイピングするための関数
def geturl(urls):
    all_text=[]
    for url in urls:
        r=requests.get(url)
        c=r.content
        soup=BeautifulSoup(c,"html.parser")
        article1_content=soup.find_all("p")
        temp=[]
        for con in article1_content:
            out=con.text
            temp.append(out)
        text=''.join(temp)
        all_text.append(text)
        sleep(1)
    # print(all_text)
    return all_text

# テキスト情報をMeCabで形態素解析し、名詞だけ抽出する関数
def mplg(article):
    word_list = ""
    m=MeCab.Tagger()
    m1=m.parse (text)
    for row in m1.split("\n"):
        word =row.split("\t")[0]#タブ区切りになっている１つ目を取り出す。ここには形態素が格納されている
        if word == "EOS":
            break
        else:
            pos = row.split("\t")[1]#タブ区切りになっている2つ目を取り出す。ここには品詞が格納されている
            slice = pos[:2]
            if slice == "名詞":
                word_list = word_list +" "+ word
    # print(word_list)
    return word_list

# 名詞の出現頻度からTF-IDF/COS類似度を算出し、テキスト情報のマッチ度を測る関数
def tfidf(word_list):
    docs = np.array(word_list)#Numpyの配列に変換する
    #単語を配列ベクトル化して、TF-IDFを計算する
    vecs = TfidfVectorizer(
                token_pattern=u'(?u)\\b\\w+\\b'#文字列長が 1 の単語を処理対象に含めることを意味する
                ).fit_transform(docs)
    vecs = vecs.toarray()
    return vecs


def cossim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

##実装
word_list=[]
texts=geturl(["https://ja.wikipedia.org/wiki/%E7%99%BD","https://ja.wikipedia.org/wiki/%E9%BB%92"])
for text in texts:
    word_list.append(mplg(text))
    
vecs=tfidf(word_list)
print(cossim(vecs[1],vecs[0]))

0.4828019697606638
